### Caso de Estudio

En este capítulo, estamos utilizando el conocido conjunto de datos Breast Cancer Wisconsin para realizar un análisis de clústeres. Originalmente, el conjunto de datos se propuso con el fin de entrenar clasificadores; sin embargo, también puede ser muy útil para un análisis de clúster no trivial. Contiene 569 registros compuestos por 32 atributos (incluyendo el diagnóstico y un número de identificación). Todos los atributos están estrictamente relacionados con propiedades biológicas y morfológicas de los tumores, pero nuestro objetivo es validar hipótesis genéricas considerando la verdad fundamental (benigna o maligna) y las propiedades estadísticas del conjunto de datos. Antes de continuar, es importante aclarar algunos puntos. El conjunto de datos es de alta dimensionalidad y los clústeres son no convexos (por lo que no podemos esperar una segmentación perfecta). Además, nuestro objetivo no es utilizar un algoritmo de agrupamiento para obtener los resultados de un clasificador; por lo tanto, la verdad fundamental debe tenerse en cuenta solo como una indicación genérica de un posible agrupamiento. El objetivo de este ejemplo es mostrar cómo realizar un breve análisis preliminar, seleccionar el número óptimo de clústeres y validar los resultados finales

In [ ]:
import pandas as pd

url='https://drive.google.com/file/d/1LG6hb0duLtnSiFhT9JWFNuo8qDxHV08W/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

dataset_columns = ['id','diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean','area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean','concave points_mean', 'symmetry_mean', 'fractal_dimension_mean','radius_se','texture_se', 'perimeter_se', 'area_se', 'smoothness_se','compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se','fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst','area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst','concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
df = pd.read_csv(url, sep="," ,index_col=0, names=bc_dataset_columns)